In [20]:
from datetime import datetime, timedelta
from netCDF4 import Dataset


import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import glob

import numpy as np
import math


from netCDF4 import Dataset

import pandas as pd
import xarray as xr

from metpy.calc import reduce_point_density
import metpy.calc as mpcalc
from metpy.plots import StationPlot, wx_code_to_numeric
from metpy.plots.wx_symbols import sky_cover, current_weather
from metpy.units import units
from metpy.interpolate import interpolate_to_grid, remove_nan_observations
from metpy.io import metar

In [2]:
UNIX_TIME = datetime(1970,1,1,0)

In [3]:
#set time to plot
start_year  = 2023
start_month = 4
start_day   = 9
start_hour  = 12
#start_min   = 0
    
    
#set plot domain
lat_min = 33
lat_max = 43
lon_min = -123
lon_max = -113
    
dt = datetime(start_year,start_month,start_day,start_hour)#,start_min)
del_t = datetime.now()-dt
del_t_days = del_t.days + (del_t.seconds/60/60/24)
print(del_t_days)

8.090439814814815


In [4]:
#get the data
import requests
import gzip
import shutil

#download and extract the data

if del_t_days>3:
    base_url = 'https://madis-data.cprk.ncep.noaa.gov/madisPublic1/data/archive/'
    url = f'{base_url}{dt:%Y}/{dt:%m}/{dt:%d}/point/raob/netcdf/{dt:%Y%m%d_%H%M}.gz'
    print(url)
    r = requests.get(url,allow_redirects=True)
    open('temp.nc.gz','wb').write(r.content)
    with gzip.open('temp.nc.gz','rb') as f_in:
        with open('temp.nc','wb') as f_out:
            shutil.copyfileobj(f_in,f_out)
    data=Dataset('temp.nc','r')

else:
    base_url = 'https://madis-data.cprk.ncep.noaa.gov/madisPublic1/data/point/raob/netcdf/'
    url = f'{base_url}{dt:%Y%m%d_%H%M}.gz'
    print(url)
    r = requests.get(url,allow_redirects=True)
    open('temp.nc.gz','wb').write(r.content)
    with gzip.open('temp.nc.gz','rb') as f_in:
        with open('temp.nc','wb') as f_out:
            shutil.copyfileobj(f_in,f_out)
    data=Dataset('temp.nc','r')

https://madis-data.cprk.ncep.noaa.gov/madisPublic1/data/archive/2023/04/09/point/raob/netcdf/20230409_1200.gz


In [149]:
time = [UNIX_TIME + timedelta(seconds=float(t)) for t in data['synTime']]
lat = data['staLat'][:]
lat = lat.filled(np.nan)
lon = data['staLon'][:]
lon = lon.filled(np.nan)
pres = data['prMan'][:]
pres = pres.filled(np.nan)
hght = data['htMan'][:]
hght = hght.filled(np.nan)
temp = data['tpMan'][:]
temp = temp.filled(np.nan)
temp[temp<100]=np.nan
temp = (temp-273.15)
dwptdp = data['tdMan'][:]
dwptdp = dwptdp.filled(np.nan)
dwptdp[dwptdp<100]=np.nan
dwptdp = (dwptdp-273.15)
wdir = data['wdMan'][:]
wdir = wdir.filled(np.nan)
wdir[wdir<0] = np.nan
wspd = data['wsMan'][:]
wspd = wspd.filled(np.nan)
wspd[wspd>250] = np.nan

name = data['staName'][:]
name = [i.tostring().decode()[:4] for i in name]
u,v = mpcalc.wind_components((wspd*units('m/s')).to('knots'),wdir*units.degree)

#u = np.abs(wspd) * np.cos(np.deg2rad(270-wdir))
#u[np.abs(u)>250] = np.nan
#v = np.abs(wspd) * np.sin(np.deg2rad(270-wdir))
#v[np.abs(v)>250] = np.nan

#u=u*units['kts']
#v=v*units['kts']
#print(u)

#df = pd.DataFrame(np.array([time[i],name[i],name[i],lon[i],lat[i],u[i],v[i],temp[i],dwptdp[i],pres[i]]),
#                      columns = ['time','id','name','lon','lat','u','v','temp','dwptdp','pres'])

print(len(name))

ds = xr.Dataset(
        coords=dict(
            id=(['id'],name)
            ),
        data_vars=dict(
            lon=(['id'],lon),
            lat=(['id'],lat),
            pressure=(['id','pres'],pres),
            temperature=(['id','pres'],temp),
            dwptdp=(['id','pres'],dwptdp),
            u=(['id','pres'],u),
            v=(['id','pres'],v)
            )
    )

#convert to pandas dataframe and remove obs with nans for lat/lon
df=ds.to_dataframe()
df=df[df['lat'].notna()]
df=df[df['lon'].notna()]


402
                  lon        lat  pressure  temperature  dwptdp          u  \
id   pres                                                                    
8209 3     -51.070000   0.050000     850.0    17.000000     NaN  -4.020057   
8202 3     -55.980000   2.480000     850.0    18.600006     NaN  -5.218597   
     3     -60.680000   2.840000     850.0    18.399994     NaN  -6.528446   
8022 3     -74.150002   4.700000     850.0          NaN     NaN        NaN   
MCSP 3     -81.699997  12.580000     850.0    16.600006     NaN -10.411713   
...               ...        ...       ...          ...     ...        ...   
CYUX 7     -81.250000  68.779999     850.0   -14.100006     NaN  -1.308071   
PABR 3    -156.789993  71.279999     850.0   -22.899994     NaN  -6.528446   
CYRB 7     -94.970001  74.699997     850.0   -17.299988     NaN   7.746772   
CWEU 10    -85.930000  80.000000     850.0   -24.099991     NaN  -2.546663   
CYLT 10    -62.349998  82.500000     850.0   -17.899994     

/tmp/ipykernel_3034822/4200025580.py:26: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  name = [i.tostring().decode()[:4] for i in name]


Subset based on vertical level